# Extractor

In [ ]:
import pandas as pd
import re
import pypdf

# 1. CONFIGURAZIONE
pdf_path = "Other/EUCS/EUCS PDF official file/EUCS_Pagine_85-159.pdf"

# 2. LETTURA PDF
print(f"Lettura del PDF: {pdf_path}")
try:
    reader = pypdf.PdfReader(pdf_path)
except FileNotFoundError:
    print(f"Errore: Il file {pdf_path} non è stato trovato.")
    exit()

pdf_text = ""

# Estrai il testo da tutte le pagine
for page_num, page in enumerate(reader.pages):
    pdf_text += page.extract_text()
    # Opzionale: print(f"  Pagina {page_num + 1} estratta")

print(f"\nTesto totale estratto: {len(pdf_text)} caratteri")

# 3. FUNZIONE DI ESTRAZIONE
def extract_controls_from_text(text):
    controls = []
    
    # Split basato sull'intestazione della tabella dei requisiti tipica dell'EUCS
    # Divide il testo in blocchi ogni volta che trova l'intestazione della tabella
    parts = re.split(r'Requirements\s+Ref\s+Description\s+Ass\.\s+Level\s*\n', text)
    
    if len(parts) < 2:
        print("Attenzione: Nessuna sezione 'Requirements' trovata o formato non riconosciuto.")
        return controls
    
    # Itera attraverso i blocchi di testo successivi alle intestazioni
    for block in parts[1:]:
        lines = block.split('\n')
        
        current_ref = None
        current_description = []
        current_level = None
        
        for line in lines:
            line = line.strip()
            
            # Se la riga è vuota, salta
            if not line:
                continue
            
            # CONDIZIONI DI STOP
            # 1. Se incontriamo "Guidance elements" (fine tabella requisiti)
            # 2. Se incontriamo una nuova sezione principale (es. A.1)
            if 'Guidance elements' in line or re.match(r'^A\.\d+\s+', line):
                # Salva il controllo corrente prima di uscire dal blocco
                if current_ref:
                    controls.append({
                        'Ref': current_ref,
                        'Description': ' '.join(current_description).strip(),
                        'Ass. Level': current_level if current_level else "N/A"
                    })
                    current_ref = None # Reset per evitare duplicati
                break
            
            # Pattern per identificare un NUOVO requisito: "XXX-##.## Testo (opzionale: Level)"
            # Esempio: "OIS-01.1 Policy text... Basic"
            ref_pattern = r'^([A-Z]+-\d+\.\d+)\s+(.+?)(?:\s+(Basic|Substantial|High))?\s*$'
            match = re.match(ref_pattern, line)
            
            if match:
                # Se c'era un controllo in lavorazione, salvalo
                if current_ref:
                    controls.append({
                        'Ref': current_ref,
                        'Description': ' '.join(current_description).strip(),
                        'Ass. Level': current_level if current_level else "N/A"
                    })
                
                # Inizia nuovo controllo
                current_ref = match.group(1)
                current_description = [match.group(2)]
                current_level = match.group(3) # Può essere None se il livello è a capo
            
            # GESTIONE RIGHE SUCCESSIVE (Descrizione multiriga o Livello a capo)
            elif current_ref:
                # Caso A: La riga è SOLO il livello di assurance
                if line in ['Basic', 'Substantial', 'High']:
                    current_level = line
                
                # Caso B: Stop se incontriamo un titolo di Categoria (es. "PS-02 PHYSICAL...")
                # Questo evita di includere intestazioni di pagina o di sezione nella descrizione
                elif re.match(r'^[A-Z]+-\d+\s+[A-Z\s]+', line):
                    # Salviamo e resettiamo perché siamo usciti dalla tabella
                    controls.append({
                        'Ref': current_ref,
                        'Description': ' '.join(current_description).strip(),
                        'Ass. Level': current_level if current_level else "N/A"
                    })
                    current_ref = None
                    break
                
                # Caso C: Continuazione della descrizione
                else:
                    # Escludi numeri di pagina o intestazioni ripetute se necessario
                    if "EUCS" not in line and not line.isdigit():
                        current_description.append(line)
        
        # Salva l'ultimo controllo del blocco se rimasto appeso
        if current_ref:
            controls.append({
                'Ref': current_ref,
                'Description': ' '.join(current_description).strip(),
                'Ass. Level': current_level if current_level else "N/A"
            })
    
    return controls

# 4. ESECUZIONE
print("\nEstrazione dei controlli EUCS in corso...")
controls = extract_controls_from_text(pdf_text)

# Crea il DataFrame
df = pd.DataFrame(controls)

# Pulizia finale (rimuovi duplicati esatti se presenti)
if not df.empty:
    df = df.drop_duplicates()

print(f"\n✓ Controlli estratti: {len(df)}")

if not df.empty:
    print("\nPrime 5 righe:")
    print(df.head(5).to_string())

    # 5. SALVATAGGIO
    output_file = 'Other/EUCS/EUCS_controls.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n✓ File salvato: {output_file}")

    # 6. STATISTICHE
    print(f"\n--- Statistiche ---")
    print(f"Totale controlli: {len(df)}")
    
    print(f"\nDistribuzione per livello di assurance:")
    print(df['Ass. Level'].value_counts().sort_index())

    print(f"\nControlli per categoria:")
    # Estrae la parte alfabetica prima del trattino (es. OIS da OIS-01.1)
    df['Categoria'] = df['Ref'].str.extract(r'^([A-Z]+)')[0]
    print(df['Categoria'].value_counts().sort_index())
else:
    print("\nNessun controllo estratto. Verifica il PDF o la logica di parsing.")

Lettura del PDF: Other/EUCS PDF official file/EUCS_Pagine_85-159.pdf

Testo totale estratto: 171590 caratteri

Estrazione dei controlli EUCS in corso...

✓ Controlli estratti: 533

Prime 5 righe:
        Ref                                                                                                                                                                                                           Description   Ass. Level
0  OIS-01.1  The CSP shall define, implement, maintain and continually improve an information security management system (ISMS), covering at least the operational units, locations and processes for providing the cloud service        Basic
1  OIS-01.2                                                                                                                                                                      The ISMS shall be in accordance to ISO/IEC 27001  Substantial
2  OIS-01.3                                                                            

### Some controls are N/A in the ass level so i need to fix it manually

## Merge the extracted text into the eucs file without text from emerald

In [ ]:
import pandas as pd
import re

# Leggi i due file
print("Lettura dei file CSV...")
eucs_requirements = pd.read_csv('Other/EUCS/NewEucsRequirementsOriginalInakiFile.csv')
eucs_controls = pd.read_csv('Other/EUCS/EUCS_control.csv')

print(f"File NewEucsRequirements.csv: {len(eucs_requirements)} righe")
print(f"File EUCS_controls.csv: {len(eucs_controls)} righe")

# Pulisci la colonna 'EUCS Control (2022)' per estrarre il codice
def extract_control_code(control_str):
    """Estrae il codice del controllo dalla stringa"""
    if pd.isna(control_str):
        return None
    match = re.match(r'^([A-Z]+-\d+)\s*', str(control_str).strip())
    return match.group(1) if match else None

# Crea una colonna con il codice estratto dal file originale
eucs_requirements['Control_Code'] = eucs_requirements['EUCS Control (2022)'].apply(extract_control_code)

print("\nCodici estratti dal file NewEucsRequirements.csv:")
print(eucs_requirements[['EUCS Control (2022)', 'Control_Code']].head(10))

# Estrai il codice principale dai ref dettagliati (es: OIS-01 da OIS-01.1)
eucs_controls['Control_Code'] = eucs_controls['Ref'].str.extract(r'^([A-Z]+-\d+)')[0]

print("\nCodici dal file EUCS_controls.csv:")
print(eucs_controls[['Ref', 'Control_Code']].head(10))

# Crea le righe duplicate con i dettagli
merged_data = []

for idx, req_row in eucs_requirements.iterrows():
    control_code = req_row['Control_Code']
    
    # Trova tutti i controlli dettagliati correlati
    matching_controls = eucs_controls[eucs_controls['Control_Code'] == control_code]
    
    if not matching_controls.empty:
        # Per ogni sotto-controllo trovato, crea una riga
        for ctrl_idx, ctrl_row in matching_controls.iterrows():
            merged_row = req_row.to_dict()
            merged_row['EUCS Text'] = ctrl_row['Description']
            merged_row['EUCS Ref (Detailed)'] = ctrl_row['Ref']
            merged_row['EUCS Ass. Level'] = ctrl_row['Ass. Level']
            merged_data.append(merged_row)
    else:
        # Se non trova corrispondenze, mantieni la riga com'è
        merged_row = req_row.to_dict()
        merged_row['EUCS Text'] = ''
        merged_row['EUCS Ref (Detailed)'] = ''
        merged_row['EUCS Ass. Level'] = ''
        merged_data.append(merged_row)

# Crea il DataFrame merged
df_merged = pd.DataFrame(merged_data)

# Ordina le colonne in modo logico
cols_order = [
    'EUCS Category',
    'EUCS Control (2022)',
    'Control_Code',
    'EUCS Ref (Detailed)',
    'EUCS Text',
    'EUCS Ass. Level',
    'Code',
    'C5.2020 GERMANY',
    'SecNumCloud FRANCE',
    'ISO 27002',
    'ISO 27017'
]

# Mantieni solo le colonne che esistono
cols_order = [col for col in cols_order if col in df_merged.columns]
df_merged = df_merged[cols_order]

print(f"\nFile merged: {len(df_merged)} righe (da {len(eucs_requirements)} originali)")
print("\nAnteprima del risultato:")
print(df_merged.head(15).to_string())

# Salva il file merged
output_file = 'Schemes/NewEucsRequirements_with_texts.csv'
df_merged.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✓ File salvato: {output_file}")

# Statistiche
print(f"\nStatistiche:")
print(f"Righe originali: {len(eucs_requirements)}")
print(f"Righe nel file merged: {len(df_merged)}")
print(f"Righe con testo EUCS: {(df_merged['EUCS Text'] != '').sum()}")
print(f"Righe senza testo (NO MATCH): {(df_merged['EUCS Text'] == '').sum()}")

# Dettaglio dei NO MATCH
no_match_rows = df_merged[df_merged['EUCS Text'] == '']
print(f"\n⚠️  Righe che NON hanno trovato match ({len(no_match_rows)}):")
if len(no_match_rows) > 0:
    for idx, row in no_match_rows.iterrows():
        print(f"   - {row['EUCS Control (2022)']} (Control_Code: {row['Control_Code']})")
else:
    print("   Nessuna riga senza match! ✓")

# Statistiche per categoria
print(f"\nRighe per EUCS Category:")
print(df_merged['EUCS Category'].value_counts().sort_index())

# Visualizza campioni
print("\nCampioni di righe duplicate:")
sample_control = df_merged[df_merged['EUCS Text'] != ''].iloc[0]['EUCS Control (2022)']
sample_rows = df_merged[df_merged['EUCS Control (2022)'] == sample_control]
print(f"\nControllo: {sample_control} ({len(sample_rows)} righe)")
for idx, row in sample_rows.iterrows():
    print(f"  → {row['EUCS Ref (Detailed)']}: {row['EUCS Text'][:80]}... [{row['EUCS Ass. Level']}]")

Lettura dei file CSV...
File NewEucsRequirements.csv: 119 righe
File EUCS_controls.csv: 533 righe

Codici estratti dal file NewEucsRequirements.csv:
                                 EUCS Control (2022) Control_Code
0   OIS-01  - INFORMATION SECURITY MANAGEMENT SYSTEM       OIS-01
1                    OIS-02  - SEGREGATION OF DUTIES       OIS-02
2  OIS-03  - CONTACT WITH AUTHORITIES AND INTERES...       OIS-03
3  OIS-04 - INFORMATION SECURITY IN PROJECT MANAG...       OIS-04
4       ISP-01  - GLOBAL INFORMATION SECURITY POLICY       ISP-01
5          ISP-02 - SECURITY POLICIES AND PROCEDURES       ISP-02
6                               ISP-03  - EXCEPTIONS       ISP-03
7                    RM-01  - RISK MANAGEMENT POLICY        RM-01
8            RM-02  - RISK ASSESSMENT IMPLEMENTATION        RM-02
9             RM-03  - RISK TREATMENT IMPLEMENTATION        RM-03

Codici dal file EUCS_controls.csv:
        Ref Control_Code
0  OIS-01.1       OIS-01
1  OIS-01.2       OIS-01
2  OIS-01.3   

⚠️  Righe che NON hanno trovato match (3):
   - OPS-22  - SEPARATION OF DATASETS IN THE CLOUD INFRASTRUCTURE (Control_Code: OPS-22)
   - CS-02  - SECURITY REQUIREMENTS TO CONNECT WITHIN THE CSP’S NETWORK (Control_Code: CS-02)
   - DEV-08 CONTROLLING EXCHANGES WITH SUPPLIERS OF FUNCTIONAL COMPONENTS (Control_Code: DEV-08)
